In [53]:
# TODO Transfer learning & Main architecture
#     Lab5 - task2 dice 0.95    
#     model  saved in ‘/home/k8s-group5’
#     Load data function (try Mehdi’s) no shuffling
#     UNET 2D

# Model for transfer learning input (model, '/home/k8s-group5')

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SpatialDropout2D,Conv2DTranspose,Concatenate

import os

import tensorflow as tf
from tensorflow import keras


2.9.0


In [50]:
model_dir = '/home/k8s-group5/assets/'

def TL_unet_model(input_shape):

    input_shape = input_shape
    base = model_dir  #TODO change to the actual model, how do I import as model?

    # base = VGG16(include_top = False, 
    #                weights = "imagenet", 
    #                input_shape = input_shape)


    # freezing all layers in VGG16 
    for layer in base.layers: 
        layer.trainable = False

    # the bridge (exclude the last maxpooling layer in VGG16) 
    bridge = base.get_layer("block5_conv3").output #TODO change layer
    print(bridge.shape)

    # Decoder now
    up1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(bridge)
    print(up1.shape)
    concat_1 = Concatenate([up1, base.get_layer("block4_conv3").output], axis=3) #TODO change layer
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(concat_1)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    print(up2.shape)
    concat_2 = Concatenate([up2, base.get_layer("block3_conv3").output], axis=3) #TODO change layer
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(concat_2)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    print(up3.shape)
    concat_3 = Concatenate([up3, base.get_layer("block2_conv2").output], axis=3) #TODO change layer
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat_3)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    print(up4.shape)
    concat_4 = Concatenate([up4, base.get_layer("block1_conv2").output], axis=3) #TODO change layer
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat_4)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    print(conv10.shape)

    model_ = Model(inputs=[base.input], outputs=[conv10])

    return model_

In [44]:
def generator(x_train, y_train, batch_size):
    n_train_sample = len(x_train)
    while True:
               
        for ind in (range(0, n_train_sample, batch_size)):
            
            batch_img = x_train[ind:ind+batch_size]
            batch_label = y_train[ind:ind+batch_size]
            
            # Sanity check assures batch size always satisfied
            # by repeating the last 2-3 images at last batch.
            length = len(batch_img)
            if length == batch_size:
                pass
            else:
                for tmp in range(batch_size - length):
                    batch_img = np.append(batch_img, np.expand_dims(batch_img[-1],axis=0), axis = 0)
                    batch_label = np.append(batch_label, np.expand_dims(batch_label[-1], axis=0), axis = 0)
        
            backgound_value = x_train.min()
            data_gen_args = dict(rotation_range=10.,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     cval = backgound_value,
                                     zoom_range=0.2,
                                     horizontal_flip = True)
            
            image_datagen = ImageDataGenerator(**data_gen_args)
            mask_datagen = ImageDataGenerator(**data_gen_args)
            
            image_generator = image_datagen.flow(batch_img, shuffle=False,
                                                 batch_size=batch_size,
                                                 seed=1)
            
            mask_generator = mask_datagen.flow(batch_label, shuffle=False,
                                               batch_size=batch_size,
                                               seed=1)
            
            image = image_generator.next()
            label = mask_generator.next()
            
            yield image, label

In [52]:
input_shape = shape=(100, 100, 1)
batch_size = 8

model = TL_unet_model(input_shape)
model.summary()
train_generator = generator(image_train, mask_train, batch_size)
val_generator = generator(image_val, mask_val, batch_size)


AttributeError: 'str' object has no attribute 'layers'